In [2]:
import json
import time
import datetime

In [26]:
import vk
import numpy as np
import pandas as pd
import user_info as ui
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
SESSION = vk.AuthSession(app_id=ui.APP_ID, user_login=ui.USER_LOGIN, user_password=ui.USER_PASSWORD)
API = vk.API(SESSION)

In [5]:
def get_group_id(groups_list):
    """
    Получение id группы по названию
    :param groups_list: <list>
    :return: <list>
    """
    for group in groups_list:
        for item in API.groups.search(q=group['title'])[1:]:
            if group['title'] == item['name']:
                group['id'] = item['gid']
    return groups_list

In [6]:
def get_members(group_id):
    """
    Получение пола и возраста пользователей группы
    :param group_id: <int>
    :return: <list>
    """
    members = []
    offset = 0
    while True:
        response = API.groups.getMembers(group_id=group_id, offset=offset, fields=['sex', 'bdate'])
        members.extend(response['users'])
        offset += 1000
        # Создание задержки из-за ограничения на количество запросов в секунду
        if offset >= 3000:
            time.sleep(0.34)
        if offset >= response['count']:
            break
    return members

In [7]:
def gender_count(members_list):
    """
    Разделение пользователей группы по полам
    :param members_list: <list>
    :return: <dict>
    """
    genders = {'Female': 0, 'Male': 0}
    for member in members_list:
        if member['sex'] == 1:
            genders['Female'] += 1
        elif member['sex'] == 2:
            genders['Male'] += 1
    return genders

In [8]:
def get_year(date):
    """
    Получение года из даты рождения
    :param date: <str>
    :return: <str> or <None>
    """
    if date.count('.') == 2:
        return date[-4:]

In [35]:
def age_count(members_list):
    """
    Разделение пользователей группы по годам
    :param members_list: <list>
    :return: <dict>
    """
    current_year = datetime.datetime.now().year
    ages = {}
    for member in members_list:
        if 'bdate' in member:
            year = get_year(member['bdate'])
            if year is not None:
                age = current_year - int(year)
                ages.setdefault(age, 0)
                ages[age] += 1
    return ages

In [10]:
with open('top100.json') as file:
    groups = get_group_id(json.load(file)[0:5])

In [36]:
group_number = 1
for group in groups:
    with open('group{}.json'.format(str(group_number)), 'w') as file:
        group_info = []
        members = get_members(group['id'])
        genders = gender_count(members)
        ages = age_count(members)
        group_info.append({'name': group['title']})
        group_info.append(genders)
        group_info.append(ages)
        json.dump(group_info, file, indent=2, ensure_ascii=False)
    group_number += 1

In [ ]:
# Во избежании копипасты: для каждой из групп необходимо выполнить
# следующие три ячейки, меняя только номер группы в названии файла.
# В принципе, можно даже не давать номера переменным, а использовать
# одни и те же, если нет необходимости в доступе к каждому датафрейму.

In [37]:
with open('group1.json') as file:
    group1 = json.load(file)
    genders1 = pd.DataFrame(list(group1[1].items()), columns=['Gender', 'Count'])
    ages1 = pd.DataFrame(list(group1[2].items()), columns=['Age', 'Count'])

In [ ]:
genders1.plot(kind='bar', title ='Распределение полов', x='Gender', legend=False)

In [ ]:
ages1.sort_values(by='Age').plot(kind='bar', title ='Возраст', x='Age', legend=False)